In [17]:
import numpy as np
import torch
import math 
from torch.utils.data import Dataset


def calculate_fire_spread_rate(wind_speed_at_10m, moisture_content, slope_angle, fuel_load):
    """
    Calculate the rate of spread of a forest fire.

    Parameters:
    wind_speed_at_10m (float): Wind speed at 10 meters height in km/hr.
    moisture_content (float): Moisture content in percentage (35% - 65%).
    slope_angle (float): Slope angle in degrees.
    fuel_load (float): Fuel load in tonnes/hectare (< 6 mm diameter), default is 2 tonnes/hectare.

    Returns:
    float: Rate of spread of the fire in unspecified units.
    """
    # Constants
    a = 1.674
    b = 0.1798
    c = 0.22
    d = 0.158
    e = -0.227
    f = 0.0662

    # Convert wind speed at 10m to wind speed at 1.5m
    wind_speed_at_1_5m = a + b * wind_speed_at_10m

    # Calculate the rate of spread on flat ground
    rate_flat_ground = c * fuel_load * math.exp((wind_speed_at_1_5m ** d) * (moisture_content ** e))

    # Adjust rate for slope
    rate_on_slope = rate_flat_ground * math.exp((slope_angle ** f))

    return rate_on_slope

# Example usage
spread_rate = calculate_fire_spread_rate(20, 50, 10, 2)
spread_rate

2.4078009424508124

In [18]:
from scipy.stats import qmc
import pandas as pd


# Define the bounds for each parameter
bounds = np.array([[11.5, 34.1],  # Wind speed
                   [0, 6],        # Fuel load
                   [35, 65],      # Moisture content
                   [0, 30]])      # Slope angle

# Using Latin Hypercube Sampling (LHS) for a more structured approach
sampler = qmc.LatinHypercube(d=4)
sample = sampler.random(n=40000)
sample = qmc.scale(sample, bounds[:, 0], bounds[:, 1])

# Extracting the samples for each parameter
wind_speed_samples = sample[:, 0]
fuel_samples = sample[:, 1]
moisture_content_samples = sample[:, 2]
slope_angle_samples = sample[:, 3]

# Calculating spread rates for the sampled data
spread_rates = [calculate_fire_spread_rate(wind_speed, moisture, slope, fuel) 
                for wind_speed, moisture, slope, fuel in zip(wind_speed_samples, moisture_content_samples, slope_angle_samples, fuel_samples)]

# Creating a DataFrame to hold the sampled data and calculated spread rates
data_lhs = pd.DataFrame({
    'Wind Speed (km/hr)': wind_speed_samples,
    'Fuel Load (tonnes/hectare)': fuel_samples,
    'Moisture Content (%)': moisture_content_samples,
    'Slope Angle (degrees)': slope_angle_samples,
    'Spread Rate': spread_rates
})

data_lhs.head()

,Wind Speed (km/hr),Fuel Load (tonnes/hectare),Moisture Content (%),Slope Angle (degrees),Spread Rate
0,30.563375,5.624219,35.289715,25.103661,7.835877
1,30.450264,0.718370,49.098727,12.207938,0.903943
2,27.347398,4.375063,35.060292,27.366743,6.095250
3,12.248680,1.850334,62.038588,2.014380,1.885172
4,28.502253,3.868624,63.662320,11.511565,4.672144


In [19]:

import torch
from torch.utils.data import Dataset

class FireSpreadDataset(Dataset):
    def __init__(self, dataframe):
        # Convert the dataframe to a PyTorch tensor
        self.data = torch.tensor(dataframe.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Assuming the last column is the target variable (spread rate)
        features = self.data[idx, :-1]
        target = self.data[idx, -1]
        return features, target

# Convert the DataFrame to a PyTorch Dataset
fire_spread_data = FireSpreadDataset(data_lhs)

# Example: accessing the first item in the dataset
first_sample_features, first_sample_target = fire_spread_data[0]
print(first_sample_features, first_sample_target)


tensor([30.5634,  5.6242, 35.2897, 25.1037]) tensor(7.8359)


In [21]:
import numpy as np
import torch
from torch.utils.data import Dataset
from typing import List 
import torch.nn as nn
import torch.optim as optim

class BushfireModel(nn.Module): 
    def __init__(self, n: int, hidden_layers: List[int]): 
        super().__init__() #Inherit from the nn.module
        self.n = n # n input features
        self.hidden_layers = hidden_layers # hidden layers are the middle ones

        self.layers = nn.ModuleList([
            nn.Linear(in_dim, out_dim)
            for in_dim, out_dim in zip([n, *hidden_layers], [*hidden_layers, 1])
        ]) # Single output

    def forward(self, x: torch.tensor):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i != len(self.layers) - 1:    #We don't want to ReLU the last layer
                x = nn.functional.relu(x)  
        return x # Linear output for regression

In [22]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

# Splitting the dataset
total_size = len(fire_spread_data)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # 20% for validation
train_dataset, val_dataset = random_split(fire_spread_data, [train_size, val_size])

# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Training function
def train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()  # Training mode
        for i, (features, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(features)
            loss = loss_criterion(outputs, labels.unsqueeze(1))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()  # Validation mode
        with torch.no_grad():
            val_loss = 0
            for features, labels in val_loader:
                outputs = model(features)
                val_loss += loss_criterion(outputs, labels.unsqueeze(1)).item()

        val_loss /= len(val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

# Initialize the model
model = BushfireModel(n=4, hidden_layers=[200,100])
loss_criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# Train the model
num_epochs = 50
train_model(model, train_loader, val_loader, loss_criterion, optimizer, num_epochs)


Epoch [1/50], Loss: 4.1964, Val Loss: 4.6359
Epoch [2/50], Loss: 4.3510, Val Loss: 3.8894
Epoch [3/50], Loss: 3.2410, Val Loss: 3.1594
Epoch [4/50], Loss: 2.5266, Val Loss: 2.4050
Epoch [5/50], Loss: 1.8241, Val Loss: 1.6524
Epoch [6/50], Loss: 1.0697, Val Loss: 1.0059
Epoch [7/50], Loss: 0.5219, Val Loss: 0.5253
Epoch [8/50], Loss: 0.1773, Val Loss: 0.2272
Epoch [9/50], Loss: 0.0775, Val Loss: 0.0850
Epoch [10/50], Loss: 0.0300, Val Loss: 0.0352
Epoch [11/50], Loss: 0.0116, Val Loss: 0.0211
Epoch [12/50], Loss: 0.0306, Val Loss: 0.0165
Epoch [13/50], Loss: 0.0108, Val Loss: 0.0140
Epoch [14/50], Loss: 0.0118, Val Loss: 0.0122
Epoch [15/50], Loss: 0.0095, Val Loss: 0.0109
Epoch [16/50], Loss: 0.0065, Val Loss: 0.0094
Epoch [17/50], Loss: 0.0066, Val Loss: 0.0081
Epoch [18/50], Loss: 0.0186, Val Loss: 0.0072
Epoch [19/50], Loss: 0.0044, Val Loss: 0.0066
Epoch [20/50], Loss: 0.0072, Val Loss: 0.0060
Epoch [21/50], Loss: 0.0036, Val Loss: 0.0060
Epoch [22/50], Loss: 0.0043, Val Loss: 0.00

In [23]:
def predict(model, input_features):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        predictions = model(input_features)
    return predictions

# Example usage with one sample from the validation set
features, labels = next(iter(val_loader))
prediction = predict(model, features)
print(f'Predicted: {prediction}, Actual: {labels}')


Predicted: tensor([[1.8368],
        [7.2944],
        [3.4258],
        [5.3036],
        [7.1170],
        [0.5083],
        [5.8882],
        [6.1435],
        [3.8227],
        [1.2987],
        [2.6134],
        [2.1021],
        [4.5084],
        [6.1072],
        [6.9918],
        [2.5054],
        [2.6434],
        [5.3602],
        [1.7193],
        [6.9016],
        [6.4473],
        [4.8508],
        [4.8121],
        [5.9286],
        [1.0280],
        [6.9925],
        [3.8549],
        [2.5346],
        [3.6347],
        [5.1543],
        [6.6780],
        [0.9985],
        [0.8200],
        [6.9721],
        [6.6269],
        [2.0206],
        [5.6077],
        [3.2356],
        [7.8933],
        [5.2012],
        [2.6680],
        [5.0692],
        [6.3512],
        [3.7859],
        [2.8536],
        [3.0264],
        [1.7153],
        [6.4385],
        [2.8308],
        [6.2995],
        [7.1219],
        [4.7041],
        [4.8770],
        [1.1515],
        [2.4446],